<a href="https://colab.research.google.com/github/ahmadmasood43/Glioblastoma-Detection-and-VR-Conversion/blob/main/Classification_of_brain_tumors_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split

Import google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load and Preprocess data

In [11]:
import os

# Load tumor CSV file
tumor_data = pd.read_csv('/content/gdrive/MyDrive/new_fyp_dataset_folder/annotation_fyp.csv')

# Load non-tumor CSV file
non_tumor_data = pd.read_csv('/content/gdrive/MyDrive/new_fyp_dataset_folder/annotation_fyp_no.csv')

# Preprocess tumor data
tumor_image_dir = '/content/gdrive/MyDrive/new_fyp_dataset_folder/yes'  # Directory containing the tumor images
tumor_image_paths = []
tumor_labels = []

for _, row in tumor_data.iterrows():
    image_path = os.path.join(tumor_image_dir, row['filename'])
    tumor_image_paths.append(image_path)
    tumor_labels.append(1)  # Tumor label

# Preprocess non-tumor data
non_tumor_image_dir = '/content/gdrive/MyDrive/new_fyp_dataset_folder/no'  # Directory containing the non-tumor images
non_tumor_image_paths = []
non_tumor_labels = []

for _, row in non_tumor_data.iterrows():
    image_path = os.path.join(non_tumor_image_dir, row['filename'])
    non_tumor_image_paths.append(image_path)
    non_tumor_labels.append(0)  # Non-tumor label

# Combine tumor and non-tumor data
image_paths = tumor_image_paths + non_tumor_image_paths
labels = tumor_labels + non_tumor_labels

# Create a new DataFrame with image paths and labels
preprocessed_data = pd.DataFrame({'image_paths': image_paths, 'labels': labels})

# Convert labels to strings
preprocessed_data['labels'] = preprocessed_data['labels'].astype(str)

# Split the data into training and validation sets
train_data, valid_data = train_test_split(preprocessed_data, test_size=0.2, random_state=42)

# Preprocess data
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='image_paths',
    y_col='labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_data,
    x_col='image_paths',
    y_col='labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 683 validated image filenames belonging to 2 classes.
Found 171 validated image filenames belonging to 2 classes.


Load VGG16 model

In [12]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers except the last 2
for layer in base_model.layers[:-2]:
    layer.trainable = False


In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


Custom Classifier

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)


Epoch 1/10
22/22 [==============================] - 590s 26s/step - loss: 0.5536 - accuracy: 0.7145 - val_loss: 0.3461 - val_accuracy: 0.8070
Epoch 2/10
22/22 [==============================] - 623s 28s/step - loss: 0.3307 - accuracy: 0.8463 - val_loss: 0.2792 - val_accuracy: 0.8421
Epoch 3/10
22/22 [==============================] - 608s 28s/step - loss: 0.2430 - accuracy: 0.8990 - val_loss: 0.3036 - val_accuracy: 0.8655
Epoch 4/10
22/22 [==============================] - 576s 26s/step - loss: 0.1929 - accuracy: 0.9209 - val_loss: 0.2309 - val_accuracy: 0.8830
Epoch 5/10
22/22 [==============================] - 606s 28s/step - loss: 0.1700 - accuracy: 0.9414 - val_loss: 0.1688 - val_accuracy: 0.9532
Epoch 6/10
22/22 [==============================] - 572s 26s/step - loss: 0.1345 - accuracy: 0.9575 - val_loss: 0.1741 - val_accuracy: 0.9006
Epoch 7/10
22/22 [==============================] - 605s 28s/step - loss: 0.1135 - accuracy: 0.9561 - val_loss: 0.1271 - val_accuracy: 0.9591
Epoch 

Testing Dataset

In [25]:
import os
import cv2
import numpy as np

# Path to the folder containing the images for testing
test_folder = '/content/gdrive/MyDrive/new_fyp_dataset_folder/testing'

# Get the list of image filenames in the test folder
image_filenames = os.listdir(test_folder)

# Define the input size expected by the model
image_width = 224
image_height = 224

# Define the selected threshold value
threshold = 0.5

# Assuming you have the model object available after training
model = model # Replace with the actual name of your trained model object

# Iterate over the images and perform predictions
for image_filename in image_filenames:
    image_path = os.path.join(test_folder, image_filename)
    
    # Load the image
    try:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB if necessary
        image = cv2.resize(image, (image_width, image_height))  # Resize the image to match the input size of your model
    except Exception as e:
        print(f"Error processing image '{image_filename}': {str(e)}")
        continue
    
    # Preprocess the image
    try:
        # Perform any additional preprocessing steps here if required
        
        # Normalize the image pixel values to [0, 1]
        image = image / 255.0
        
        # Reshape the image to match the input shape expected by the model
        image = np.expand_dims(image, axis=0)
    except Exception as e:
        print(f"Error preprocessing image '{image_filename}': {str(e)}")
        continue
    
    # Perform prediction using the model
    try:
        predictions = model.predict(image)
        
        # Get the predicted class (assuming single-label classification)
        predicted_class = np.argmax(predictions, axis=1).item()
        
        # Determine the prediction based on the selected threshold
        if predictions[0][predicted_class] >= threshold:
            prediction_result = True
        else:
            prediction_result = False
        
        # Print the prediction result or perform any further processing
        
        # Example: Printing the filename and prediction result
        print(f"Image: {image_filename}, Prediction Result: {prediction_result}")
    except Exception as e:
        print(f"Error predicting image '{image_filename}': {str(e)}")


1/1 [==============================] - 1s 1s/step
Image: 1-24.png, Prediction Result: False
1/1 [==============================] - 1s 1s/step
Image: 1-22.png, Prediction Result: True
1/1 [==============================] - 1s 1s/step
Image: 1-23.png, Prediction Result: False
1/1 [==============================] - 1s 715ms/step
Image: 1-21.png, Prediction Result: True
1/1 [==============================] - 1s 562ms/step
Image: 1-25.png, Prediction Result: False
1/1 [==============================] - 1s 547ms/step
Image: 1-26.png, Prediction Result: True
1/1 [==============================] - 1s 561ms/step
Image: 1-27.png, Prediction Result: False
1/1 [==============================] - 1s 561ms/step
Image: 1-04.png, Prediction Result: False
1/1 [==============================] - 1s 557ms/step
Image: 1-02.png, Prediction Result: False
1/1 [==============================] - 1s 552ms/step
Image: 1-06.png, Prediction Result: False
1/1 [==============================] - 1s 575ms/step
Image: 1-